In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


In [2]:
# %ls drive/

In [3]:
!sudo apt-get update; sudo apt-get install libgl1-mesa-glx -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease  
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,451 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,229 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,461 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2,898 kB]
Fetched 9,291 kB in 4s (2,099 kB/s)                                            
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 98 not upgraded.


In [78]:
from tensorflow import keras
import os 
import json
import numpy as np
import cv2
from tensorflow.keras.applications.vgg16 import VGG16

In [81]:
FRAME_PER_RATE = 6   # 3프레임 당 한 이미지 저장 (낮을수록 학습량 증가)
DEVIDER = 15         # LSTM 리스트 수 (낮을수록 학습량 증가)
IMG_SIZE = 112       # 영상 이미지 resize 크기 (높을수록 학습량 n^2배 증가)

src = '학습영상(표준운동)39가지/'

directory = os.listdir(src)
alist=[]
datalist=[]
framelist = []
actions=[]

base_model = VGG16(weights='imagenet', include_top=False)

In [82]:
# for i in directory:
#     label=0
#     name=os.listdir(src+i)
#     print(i.split(" ")[0] + "\r")
#     for j in name:
#         skel = os.listdir(src+i+'/'+j)
#         for k in skel:
#             count=1
#             tmpList=[]
#             if k.endswith('mp4'):
#                 cap = cv2.VideoCapture(src+i+'/'+j+'/'+k)
#                 x=0
#                 while(cap.isOpened()):
#                     ret, frame = cap.read()
#                     if (ret != True):
#                         break
#                     frame = cv2.resize(frame, dsize=(224, 224))
#                     if not x % FRAME_PER_RATE:
#                         framelist.append(frame)
#                     x+=1
#         if len(framelist)%DEVIDER:
#             framelist = framelist[:-(len(framelist)%DEVIDER)]

In [83]:
for i in directory:
    label=0
    name=os.listdir(src+i)
    print(i.split(" ")[0] + "\r")
    for j in name:
        skel = os.listdir(src+i+'/'+j)
        for k in skel:
            if k.endswith('mp4'):
                cap = cv2.VideoCapture(src+i+'/'+j+'/'+k)
                x=0
                while(cap.isOpened()): # 영상 캡쳐링 진행
                    ret, frame = cap.read()
                    if (ret != True):  # 종료 조건
                        break
                    if not x % FRAME_PER_RATE: # Frame이 FRAME_PER_RATE 마다 한 장 저장
                        frame = cv2.resize(frame, dsize=(IMG_SIZE, IMG_SIZE))
                        framelist.append(frame)
                    x+=1
                    
            if 'json' in k:
                with open(src+i+'/'+j+'/'+k,'r') as f:
                    json_data = json.load(f)
                    
                    try:
                        x = 0
                        for data in json_data['infos']:
                            xyzList = []
                            
                            if not x % FRAME_PER_RATE:
                                for xyz in data['positions']:
                                    xyzList.append(xyz['x'])
                                    xyzList.append(xyz['y'])
                                    xyzList.append(xyz['z'])
                                xyzList.append(label)

                                datalist.append(xyzList)
                            x+=1
                    except:
                        print(i, j, k)

        ##### LSTM 분리 시 DEVIDER에 나누어 떨어지는 크기로 저장(나머지는 버림)######
        
        sliceIdx = min(len(framelist), len(datalist))
        sliceIdx -= (sliceIdx%DEVIDER)
        if sliceIdx%DEVIDER:
            framelist = framelist[:sliceIdx]
            datalist = datalist[:sliceIdx]
        elif not sliceIdx%DEVIDER:
            framelist = framelist[:sliceIdx]
            datalist = datalist[:sliceIdx]
            
        ###########################################################################
        
            
        label+=1
        if not j in actions:
            actions.append(j)

표준001
표준002
표준003
표준004
표준005
표준005 20. 햄스트링 강화 20210727_110620 헴스트링 강화 우_editinfo.json
표준006
표준007
표준008
표준009
표준010
표준011
표준012


In [84]:
framelist = np.array(framelist)
datalist = np.array(datalist)
print(framelist.shape, datalist.shape)

X_train = base_model.predict(framelist) # *** 특징 추출을 통한 경량화 ***

(37530, 112, 112, 3) (37530, 97)


In [85]:
t = X_train.shape
feature_size = t[1]*t[2]*t[3]
X_train = X_train.reshape(-1, DEVIDER, feature_size) # *** 이미지: LSTM을 이용하기 위한 reshape ***
x_data = datalist.reshape(-1, DEVIDER, 97) # *** 스켈레톤: LSTM을 이용하기 위한 reshape ***

In [94]:
X_data= x_data[:,:,:-1]
labels = x_data[:,0,-1]
print(X_train.shape)
print(X_data.shape)
print(labels)

print(actions)



(2502, 15, 4608)
(2502, 15, 96)
[ 0.  0.  0. ... 38. 38. 38.]
['1. 진자운동', '10. 90도 외전 후 내회전,외회전', '11. 톱니근 펀치', '12. 동적 베어 허그', '13. 다리직거상 운동', '14. 능동보조 무릎 굴곡 신전(누운)', '15. 능동보조 무릎 굴곡 신전(앉은)', '16. 계단 스트레칭', '17. 와위 고관절 외전', '18. 무릎 굴곡 신전(누운)', '19. 무릎 굴곡 신전(앉은)', '2. 능동보조 전방굴곡', '20. 햄스트링 강화', '21. 스쿼트', '22. 스텝박스 오르기', '23. 뒤꿈치 슬라이드(누운)', '24. 뒤꿈치 슬라이드(앉은)', '25. 뒤꿈치 들기', '26. 빈자전거 운동', '27. 레그프레스', '28. 능동보조 팔꿈치 굴곡 신전', '29. 능동보조 전완부 내회전,외회전', '3. 능동보조 내회전,외회전', '30. 능동보조 손목 굴곡 신전', '31. 수부그립', '32. 팔꿈치 굴곡 신전', '33. 전완부 내회전, 외회전', '34. 손목 굴곡 신전', '35. 손목 내전, 외전', '36. 오버헤드 팔꿈치 신전', '37. 삼두근 딥스', '38. 능동보조 발목 굴곡 신전', '39. 발목 굴곡 신전', '4. 능동보조 내전', '5. 활차운동', '6. 팔꿈치 몸통 붙이기', '7. 능동 전방 굴곡', '8. 능동 내회전, 외회전', '9. 능동 외전']


In [95]:
from tensorflow.keras.utils import to_categorical
y_data = to_categorical(labels,num_classes=len(actions))

from sklearn.model_selection import train_test_split

X_data=X_data.astype(np.float32)
y_data=y_data.astype(np.float32)
print(X_train.shape,y_data.shape)
print(X_data.shape,y_data.shape)
xf_train, xf_val, yf_train, yf_val = train_test_split(X_train, y_data, test_size=0.2,random_state=2021, shuffle=True) # 이미지셋 분리
xj_train, xj_val, yj_train, yj_val = train_test_split(X_data, y_data, test_size=0.2,random_state=2021, shuffle=True)  # 스켈레톤 분리


xf_val,xf_test,yf_val ,yf_test = train_test_split(xf_val,yf_val, test_size=0.5,random_state=2022, shuffle=True) # 이미지셋 V:T 분리
xj_val,xj_test,yj_val ,yj_test = train_test_split(xj_val,yj_val, test_size=0.5,random_state=2022, shuffle=True) # 스켈레톤 V:T 분리

print('-----')
print(xf_train.shape, yf_train.shape)
print(xf_val.shape, yf_val.shape)
print(xf_test.shape, yf_test.shape)
print()
print(xj_train.shape,yj_train.shape)
print(xj_val.shape, yj_val.shape)
print(xj_test.shape, yj_test.shape)
print('-----')


(2502, 15, 4608) (2502, 39)
(2502, 15, 96) (2502, 39)
-----
(2001, 15, 4608) (2001, 39)
(250, 15, 4608) (250, 39)
(251, 15, 4608) (251, 39)

(2001, 15, 96) (2001, 39)
(250, 15, 96) (250, 39)
(251, 15, 96) (251, 39)
-----


In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, concatenate, Dropout

input1 = keras.Input(shape=xf_train.shape[1:3])  # 이미지 레이어
input2 = keras.Input(shape=xj_train.shape[1:3])  # 스켈레톤 레이어

# 이미지 레이어
x = LSTM(512, activation='relu')(input1)
x = Dropout(0.025)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.025)(x)
x = Dense(64, activation='relu')(x)
m1 = keras.Model(input1, x)


# 스켈레톤 레이어
y = LSTM(128, activation='relu')(input2)
y = Dense(64, activation='relu')(y)
m2 = keras.Model(input2, y)

# 레이어 결합
z = concatenate([m1.output, m2.output])

outputs = Dense(39, activation='softmax')(z)

# 멀티 인풋 모델 생성
model = keras.Model([input1, input2], outputs)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "functional_59"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 15, 4608)]   0                                            
__________________________________________________________________________________________________
lstm_18 (LSTM)                  (None, 512)          10487808    input_22[0][0]                   
__________________________________________________________________________________________________
dropout_18 (Dropout)            (None, 512)          0           lstm_18[0][0]                    
__________________________________________________________________________________________________
dense_36 (Dense)                (None, 128)          65664       dropout_18[0][0]                 
______________________________________________________________________________________

In [98]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# 사용법 참고: train값) in: [train1, train2] // out: train_y
# 사용법 참고: val값) ([val1, val2], val_y) 형태

history = model.fit(
    [xf_train, xj_train],
    yf_train,
    validation_data=([xf_val, xj_val], yf_val),
    epochs=100,
    callbacks=[
               ModelCheckpoint('model.h5',monitor='val_acc',verbose=1,save_best_only=True,mode='auto'),
               ReduceLROnPlateau(monitor='val_acc', factor=0.5,patience=50,verbose=1,mode='auto')
    ]
)

Epoch 1/100
63/63 [==============================] - ETA: 0s - loss: 4.3671 - acc: 0.0875
Epoch 00001: val_acc improved from -inf to 0.15600, saving model to model.h5
63/63 [==============================] - 6s 96ms/step - loss: 4.3671 - acc: 0.0875 - val_loss: 3.1423 - val_acc: 0.1560
Epoch 2/100
63/63 [==============================] - ETA: 0s - loss: 3.0205 - acc: 0.1549
Epoch 00002: val_acc improved from 0.15600 to 0.21600, saving model to model.h5
63/63 [==============================] - 6s 95ms/step - loss: 3.0205 - acc: 0.1549 - val_loss: 2.4454 - val_acc: 0.2160
Epoch 3/100
63/63 [==============================] - ETA: 0s - loss: 2.2680 - acc: 0.2419
Epoch 00003: val_acc improved from 0.21600 to 0.28000, saving model to model.h5
63/63 [==============================] - 6s 99ms/step - loss: 2.2680 - acc: 0.2419 - val_loss: 2.0840 - val_acc: 0.2800
Epoch 4/100
63/63 [==============================] - ETA: 0s - loss: 2.1334 - acc: 0.3218
Epoch 00004: val_acc improved from 0.28000 

63/63 [==============================] - ETA: 0s - loss: 0.6276 - acc: 0.7661
Epoch 00030: val_acc did not improve from 0.75600
63/63 [==============================] - 6s 94ms/step - loss: 0.6276 - acc: 0.7661 - val_loss: 0.7174 - val_acc: 0.7440
Epoch 31/100
63/63 [==============================] - ETA: 0s - loss: 0.5630 - acc: 0.7606
Epoch 00031: val_acc did not improve from 0.75600
63/63 [==============================] - 5s 86ms/step - loss: 0.5630 - acc: 0.7606 - val_loss: 0.7031 - val_acc: 0.7320
Epoch 32/100
63/63 [==============================] - ETA: 0s - loss: 0.5291 - acc: 0.7936
Epoch 00032: val_acc did not improve from 0.75600
63/63 [==============================] - 6s 89ms/step - loss: 0.5291 - acc: 0.7936 - val_loss: 0.7244 - val_acc: 0.7480
Epoch 33/100
63/63 [==============================] - ETA: 0s - loss: 0.5519 - acc: 0.7796
Epoch 00033: val_acc improved from 0.75600 to 0.76000, saving model to model.h5
63/63 [==============================] - 6s 101ms/step - lo

Epoch 61/100
63/63 [==============================] - ETA: 0s - loss: 0.3557 - acc: 0.8671
Epoch 00061: val_acc did not improve from 0.86000
63/63 [==============================] - 5s 86ms/step - loss: 0.3557 - acc: 0.8671 - val_loss: 0.5926 - val_acc: 0.7960
Epoch 62/100
63/63 [==============================] - ETA: 0s - loss: 0.3944 - acc: 0.8766
Epoch 00062: val_acc did not improve from 0.86000
63/63 [==============================] - 6s 90ms/step - loss: 0.3944 - acc: 0.8766 - val_loss: 0.4436 - val_acc: 0.8480
Epoch 63/100
63/63 [==============================] - ETA: 0s - loss: 0.3061 - acc: 0.8776
Epoch 00063: val_acc did not improve from 0.86000
63/63 [==============================] - 5s 85ms/step - loss: 0.3061 - acc: 0.8776 - val_loss: 0.7361 - val_acc: 0.7560
Epoch 64/100
63/63 [==============================] - ETA: 0s - loss: 0.3215 - acc: 0.8671
Epoch 00064: val_acc did not improve from 0.86000
63/63 [==============================] - 6s 87ms/step - loss: 0.3215 - acc: 

63/63 [==============================] - ETA: 0s - loss: 0.1873 - acc: 0.9275
Epoch 00092: val_acc did not improve from 0.89600
63/63 [==============================] - 6s 91ms/step - loss: 0.1873 - acc: 0.9275 - val_loss: 0.4569 - val_acc: 0.8640
Epoch 93/100
63/63 [==============================] - ETA: 0s - loss: 0.1858 - acc: 0.9275
Epoch 00093: val_acc did not improve from 0.89600
63/63 [==============================] - 5s 85ms/step - loss: 0.1858 - acc: 0.9275 - val_loss: 0.4875 - val_acc: 0.8440
Epoch 94/100
63/63 [==============================] - ETA: 0s - loss: 0.2135 - acc: 0.9210
Epoch 00094: val_acc did not improve from 0.89600
63/63 [==============================] - 5s 86ms/step - loss: 0.2135 - acc: 0.9210 - val_loss: 0.6336 - val_acc: 0.8040
Epoch 95/100
63/63 [==============================] - ETA: 0s - loss: 0.2468 - acc: 0.9160
Epoch 00095: val_acc did not improve from 0.89600
63/63 [==============================] - 5s 87ms/step - loss: 0.2468 - acc: 0.9160 - val_

In [27]:
import matplotlib.pyplot as plt
history.history.keys()

ModuleNotFoundError: No module named 'matplotlib.pyplot'

In [26]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

AttributeError: module 'matplotlib' has no attribute 'plot'

In [ ]:
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend();

In [115]:
model.load_weights("model.h5") # !!위험!!최고 가중치 추출: 마지막 기록학습이 날아갑니다.

In [116]:
evaluated = model.evaluate([xf_val, xj_val],yf_val,batch_size=64)
print('test loss, test acc:', evaluated)

4/4 [==============================] - 0s 15ms/step - loss: 0.3955 - acc: 0.8960
test loss, test acc: [0.395502507686615, 0.8960000276565552]


In [130]:
VideoToPred = "학습영상(표준운동)39가지/표준008/10. 90도 외전 후 내회전,외회전"
video = "20210803_151005_color.mp4"
skeleton = "20210803_151005 90도 외전 후 내회전외회전 우_frameinfo_edited.json"

frames = []
xyzs = []
#같은 전처리 과정 후 위에서 설정한 영상 및 json파일로 predict 후 영상에 대한 결과 검출
cap = cv2.VideoCapture(f"{VideoToPred}/{video}")
while(cap.isOpened()):
    ret, frame = cap.read()
    if (ret != True):
        break
    frame = cv2.resize(frame, dsize=(96, 96))
    frames.append(frame)
                       
f = open(f"{VideoToPred}/{skeleton}", "r")
json_data = json.load(f)

for data in json_data['infos']:
    xyz = []
    for p in data['positions']:
        xyz.append(p['x'])
        xyz.append(p['y'])
        xyz.append(p['z'])
    xyzs.append(xyz)

frames = np.array(frames)
xyzs = np.array(xyzs)

sliceIdx = min(len(frames), len(xyzs))
sliceIdx -= (sliceIdx%DEVIDER)
if sliceIdx%DEVIDER:
    frames = frames[:sliceIdx]
    xyzs = xyzs[:sliceIdx]
elif not sliceIdx%DEVIDER:
    frames = frames[:sliceIdx]
    xyzs = xyzs[:sliceIdx]
    
frames = base_model.predict(frames).reshape(-1, feature_size)

frames, xyzs = frames.reshape(-1, DEVIDER, feature_size), xyzs.reshape(-1, DEVIDER, 96)
    
result = model.predict([frames, xyzs])
prediction = np.argmax(result, axis=1)
pred = np.unique(prediction, return_counts=True)
# if len(pred[0])>=2:
sorter = np.argsort(pred[1])
predict = pred[0][sorter][-1]
# else: 
#     predict.append(pred[0][0])

# 파일명 순으로 정렬
dirIdx = ['1. 진자운동', '10. 90도 외전 후 내회전,외회전', '11. 톱니근 펀치', '12. 동적 베어 허그', '13. 다리직거상 운동', '14. 능동보조 무릎 굴곡 신전(누운)', '15. 능동보조 무릎 굴곡 신전(앉은)', '16. 계단 스트레칭', '17. 와위 고관절 외전', '18. 무릎 굴곡 신전(누운)', '19. 무릎 굴곡 신전(앉은)', '2. 능동보조 전방굴곡', '20. 햄스트링 강화', '21. 스쿼트', '22. 스텝박스 오르기', '23. 뒤꿈치 슬라이드(누운)', '24. 뒤꿈치 슬라이드(앉은)', '25. 뒤꿈치 들기', '26. 빈자전거 운동', '27. 레그프레스', '28. 능동보조 팔꿈치 굴곡 신전', '29. 능동보조 전완부 내회전,외회전', '3. 능동보조 내회전,외회전', '30. 능동보조 손목 굴곡 신전', '31. 수부그립', '32. 팔꿈치 굴곡 신전', '33. 전완부 내회전, 외회전', '34. 손목 굴곡 신전', '35. 손목 내전, 외전', '36. 오버헤드 팔꿈치 신전', '37. 삼두근 딥스', '38. 능동보조 발목 굴곡 신전', '39. 발목 굴곡 신전', '4. 능동보조 내전', '5. 활차운동', '6. 팔꿈치 몸통 붙이기', '7. 능동 전방 굴곡', '8. 능동 내회전, 외회전', '9. 능동 외전']
# print(prediction)
print(dirIdx[predict]) #예측 결과 도출

10. 90도 외전 후 내회전,외회전
